# 1, Imports & display settings

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import os

pd.set_option('display.max_columns', 200)
plt.rcParams['figure.figsize'] = (12,6)
sns.set_context('talk')

# 2, Load raw CSV and sanity checks

In [ ]:
raw_path = "../data/benin.csv"
if not os.path.exists(raw_path):
    raise FileNotFoundError(f"{raw_path} not found. Place raw CSV in data/")

df = pd.read_csv(raw_path)

print("shape:", df.shape)
display(df.head())
display(df.tail())
display(df.info())

# 3, Parse Timestamp, convert dtypes, and check duplicates

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

n_bad_ts = df['Timestamp'].isna().sum()
print(f"Bad/missing timestamps: {n_bad_ts}")

if n_bad_ts > 0:
    display(df[df['Timestamp'].isna()].head())

df = df.sort_values('Timestamp').reset_index(drop=True)
df = df.set_index('Timestamp')

dups = df.index.duplicated().sum()
print(f"Duplicate timestamp rows: {dups}")

# 4, Summary statistics & missing-value report

In [ ]:
num_desc = df.describe().T
display(num_desc)

missing = df.isna().sum().to_frame('n_missing')
missing['pct_missing'] = missing['n_missing'] / len(df) * 100
missing = missing.sort_values('pct_missing', ascending=False)
display(missing)

cols_above_5pct = missing[missing['pct_missing'] > 5].index.tolist()
print("Columns with >5% missing:", cols_above_5pct)

# 5, Initial cleaning: drop empty columns and obvious fixes

In [ ]:
if 'Comments' in df.columns and df['Comments'].isna().all():
    df = df.drop(columns=['Comments'])
    print("Dropped Comments column (all empty).")

if 'Cleaning' in df.columns:
    df['Cleaning'] = pd.to_numeric(df['Cleaning'], errors='coerce').fillna(0).astype(int)

# 6, Handle physically impossible values (negatives for irradiance)

In [ ]:
irr_cols = [c for c in ['GHI','DNI','DHI'] if c in df.columns]

for c in irr_cols:
    nneg = (df[c] < 0).sum()
    if nneg > 0:
        print(f"{c}: {nneg} negative values -> set to NaN")
        df.loc[df[c] < 0, c] = np.nan

# 7, Z-score outlier detection for target columns

In [ ]:
cols_to_z = [c for c in ['GHI','DNI','DHI','ModA','ModB','WS','WSgust'] if c in df.columns]

z_df = df[cols_to_z].copy()
z_scores = z_df.apply(lambda x: np.abs(stats.zscore(x.dropna())), axis=0)

outlier_mask = pd.DataFrame(False, index=df.index, columns=cols_to_z)
for c in cols_to_z:
    col = df[c]
    z = (col - col.mean())/col.std(ddof=0)
    outlier_mask.loc[z.abs() > 3, c] = True

outlier_counts = outlier_mask.sum().to_frame('n_outliers')
outlier_counts['pct_outliers'] = outlier_counts['n_outliers'] / len(df) * 100
display(outlier_counts.sort_values('pct_outliers', ascending=False))

rows_flagged = outlier_mask.any(axis=1).sum()
print(f"Rows flagged as outlier in any monitored column: {rows_flagged}")

# 8, Apply cleaning

In [ ]:
impute_cols = [c for c in ['GHI','DNI','DHI','ModA','ModB','TModA','TModB','WS','WSgust'] if c in df.columns]

for c in impute_cols:
    med = df[c].median()
    n_missing_before = df[c].isna().sum()
    df[c] = df[c].fillna(med)
    n_missing_after = df[c].isna().sum()
    print(f"{c}: median={med}; missing before={n_missing_before}, after={n_missing_after}")

multi_outliers = outlier_mask.sum(axis=1)
drop_rows = multi_outliers[multi_outliers >= 2].index
print(f"Rows with outliers in >=2 columns: {len(drop_rows)}")
if len(drop_rows) > 0 and len(drop_rows) < 5000:
    df = df.drop(index=drop_rows)
    print(f"Dropped {len(drop_rows)} rows due to multiple outlier flags.")
else:
    print("Not dropping multi-outlier rows (too many or none); they are left after imputation).")

# 9, Save cleaned data

In [ ]:
os.makedirs('data', exist_ok=True)
clean_path = "data/benin_clean.csv"
df.to_csv(clean_path, index=True)
print("Saved cleaned file to:", clean_path)

# 10, Time features and resampling for plotting

In [ ]:
df.index = pd.to_datetime(df.index)

df['hour'] = df.index.hour
df['month'] = df.index.month
df['date'] = df.index.date

expected_numeric = ['GHI','DNI','DHI','ModA','ModB','Tamb','TModA','TModB',
                    'RH','WS','WSgust','WSstdev','WD','WDstdev','BP','Precipitation']
for c in expected_numeric:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

numeric_cols = df.select_dtypes(include='number').columns
daily = df[numeric_cols].resample('D').mean()
hourly = df.groupby('hour')[numeric_cols].mean()

print("daily.shape:", daily.shape, "hourly.shape:", hourly.shape)
display(daily[['GHI','DNI','DHI','Tamb']].head())

# 11, Time series plots

In [ ]:
cols_plot = [c for c in ['GHI','DNI','DHI','Tamb'] if c in df.columns]
daily[cols_plot].plot(subplots=False, figsize=(14,5))
plt.title('Daily average: ' + ', '.join(cols_plot))
plt.ylabel('Value')
plt.show()

# 12, Diurnal (hourly) pattern

In [ ]:
hourly_cols = [c for c in ['GHI','DNI','DHI'] if c in df.columns]
hourly[hourly_cols].plot(figsize=(10,5))
plt.title('Average by hour of day')
plt.xlabel('Hour (0-23)')
plt.show()

# 13, Cleaning impact: ModA & ModB pre/post cleaning

In [ ]:
if 'Cleaning' in df.columns and 'ModA' in df.columns and 'ModB' in df.columns:
    grouped = df.groupby('Cleaning')[['ModA','ModB']].mean()
    display(grouped)
    grouped.plot(kind='bar')
    plt.title('Average ModA & ModB by Cleaning flag')
    plt.show()
else:
    print("Cleaning or module columns missing; skipping this step.")

# 14, Correlation heatmap

In [ ]:
corr_cols = [c for c in ['GHI','DNI','DHI','ModA','ModB','TModA','TModB','Tamb','RH','BP'] if c in df.columns]
plt.figure(figsize=(10,8))
sns.heatmap(df[corr_cols].corr(), annot=True, fmt=".2f", cmap='coolwarm', center=0)
plt.title('Correlation heatmap')
plt.show()

# 15, Scatter plots

In [ ]:
# WS vs GHI
if 'WS' in df.columns and 'GHI' in df.columns:
    plt.scatter(df['WS'], df['GHI'], alpha=0.2)
    plt.xlabel('WS (m/s)'); plt.ylabel('GHI (W/m2)')
    plt.title('Wind Speed vs GHI'); plt.show()

# RH vs Tamb
if 'RH' in df.columns and 'Tamb' in df.columns:
    plt.scatter(df['RH'], df['Tamb'], alpha=0.2)
    plt.xlabel('RH (%)'); plt.ylabel('Tamb (°C)')
    plt.title('Relative Humidity vs Ambient Temp'); plt.show()

# 16, Wind rose

In [ ]:
if set(['WD','WS']).issubset(df.columns):
    wd = df['WD'].dropna()
    ws = df['WS'].dropna()
    # use WD to bin directions and show counts
    wd_rad = np.deg2rad(wd)
    bins = 16
    counts, edges = np.histogram(wd_rad, bins=bins)
    angles = (edges[:-1] + edges[1:]) / 2
    ax = plt.subplot(111, polar=True)
    ax.bar(angles, counts, width=(2*np.pi)/bins)
    plt.title('Wind direction (counts) - polar histogram')
    plt.show()
else:
    print("WD or WS missing; skipping wind rose.")

# 17, Bubble chart: GHI vs Tamb (size = RH)

In [ ]:
if set(['GHI','Tamb','RH']).issubset(df.columns):
    s = (df['RH'] - df['RH'].min()) / (df['RH'].max() - df['RH'].min()) * 200 + 10
    plt.scatter(df['Tamb'], df['GHI'], s=s, alpha=0.1)
    plt.xlabel('Tamb (°C)'); plt.ylabel('GHI (W/m2)')
    plt.title('GHI vs Tamb (bubble size ~ RH)')
    plt.show()
else:
    print("GHI/Tamb/RH missing; skipping bubble chart.")

# 18, Export summary tables

In [ ]:
summary = df[['GHI','DNI','DHI']].agg(['mean', 'median', 'std']).T
summary.to_csv('data/benin_summary_table.csv')
display(summary)